# Example: Using QR iteration to compute eigenvalues and eigenvectors
This example will familiarize students with computing the [eigenvalues and eigenvectors]() of a real matrix $\mathbf{A}\in\mathbb{R}^{n\times{n}}$ using [the QR iteration algorithm](https://en.wikipedia.org/wiki/QR_algorithm).

## Setup
This example requires several external libraries and a function to compute the outer product. Let's download and install these packages and call our `Include.jl` file.

In [3]:
include("Include.jl");

  Activating project at `~/Desktop/julia_work/CHEME-4800-5800-Examples-Fall-2024/lecture/week-7/L7c`
   Installed Statistics ─ v1.11.1
    Updating `~/Desktop/julia_work/CHEME-4800-5800-Examples-Fall-2024/lecture/week-7/L7c/Project.toml`
  [6e4b80f9] + BenchmarkTools v1.5.0
  [8bb1440f] + DelimitedFiles v1.9.1
  [28b8d3ca] + GR v0.73.7
  [82e4d734] + ImageIO v0.6.8
⌃ [6218d12a] + ImageMagick v1.2.1
  [916415d5] + Images v0.26.1
  [91a5bcdd] + Plots v1.40.8
  [5e47fb64] + TestImages v1.8.0
  [37e2e46d] ~ LinearAlgebra ⇒ v1.11.0
    Updating `~/Desktop/julia_work/CHEME-4800-5800-Examples-Fall-2024/lecture/week-7/L7c/Manifest.toml`
  [621f4979] + AbstractFFTs v1.5.0
  [79e6a3ab] + Adapt v4.0.4
  [ec485272] + ArnoldiMethod v0.4.0
  [4fba245c] + ArrayInterface v7.16.0
  [13072b0f] + AxisAlgorithms v1.1.0
  [39de3d68] + AxisArrays v0.4.7
  [6e4b80f9] + BenchmarkTools v1.5.0
  [d1d4a3ce] + BitFlags v0.1.9
  [62783981] + BitTwiddlingConvenienceFunctions v0.1.6
  [fa961155] + CEnum v0.5.0
  [2a

## Compute eigenvalues and eigenvectors
Let's do an example where we compute the eigenvalues and eigenvectors of a square matrix $\mathbf{A}$ using our implementation of the [QR iteration algorithm](https://en.wikipedia.org/wiki/QR_algorithm), which is in [src/Compute.jl](src/Compute.jl). Compute the eigenvalues and eigenvectors of the matrix:

$$
\mathbf{A} = \begin{bmatrix}
3.0 & -0.3 & -0.2 \\
0.1 & 7.0 & -0.3 \\
0.3 & -0.2 & 10.0 \\
\end{bmatrix}
$$

How well does our answer compare to the values generated by [eigen function](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.eigen), which is part of the Julia standard library?

In [38]:
# Setup matrix the n x n matrix A (n = 3)
A = [3.0 -0.3 -0.2 ; 0.1 7.0 -0.3 ; 0.3 -0.2 10.0]

3×3 Matrix{Float64}:
 3.0  -0.3  -0.2
 0.1   7.0  -0.3
 0.3  -0.2  10.0

In [36]:
A[2,3]

-0.3

First, let's use the [built-in eigen function](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.eigen) and see what we get. The [eigen function](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.eigen) takes a square array `A` as an argument and returns the eigendecomposition:

In [7]:
# Decompose using the built-in function
F = eigen(A);   # eigenvalues and vectors in F of type Eigen
λ = F.values;   # vector of eigenvalues
V = F.vectors;  # 3 x 3 matrix of eigenvectors, each col is an eigenvector

In [40]:
λ

3-element Vector{Float64}:
  3.017277143754452
  6.9699146114784165
 10.012808244767134

In [8]:
V

3×3 Matrix{Float64}:
 -0.998641    0.0788277  -0.024097
  0.0283674  -0.994181   -0.099848
  0.0437173  -0.0734251   0.994711

Next, let's call the `qriteration(...)` function encoded in [src/Compute.jl](src/Compute.jl) and see what happens:

In [10]:
# Call our qriteration function (assumed to be included in the workspace)
(λ̂,V̂) = qriteration(A; maxiter=10000, tolerance=1e-6);

Finally, let's compare our computed values with the values calculated using the builtin function using the [norm function](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.norm) exported by the `LinearAlgebra` package.
* A norm is a function from a real or complex vector space to the non-negative real numbers that computes the distance between two mathematical objects 

#### Check: Eigenvalues
Let's check if the eigenvalues computed by the [`eigen(...)` function](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.eigen) are the same as the ones we just calculated [by our `qriteration(...)` implementation](src/Compute.jl).

In [13]:
[λ λ̂] # eigen is the first col, our is the second col

3×2 Matrix{Float64}:
  3.01728   3.01728
  6.96991   6.96991
 10.0128   10.0128

__What is a norm?__ A [norm function](https://en.wikipedia.org/wiki/Norm_(mathematics)) is an abstraction of a distance function. It measures (somehow) the magnitude of a vector or a matrix. In Julia, we [use the builtin `norm(...)` function](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.norm)

In [15]:
norm(λ - λ̂)

2.2554654274383627e-6

Alternatively, we could use the [isapprox function](https://docs.julialang.org/en/v1/base/math/#Base.isapprox) to check if $\lambda$ and $\hat{\lambda}$ are `close` in some relative (`rtol`) or absolute (`atol`) sense:

In [17]:
isapprox.(λ̂,λ, atol=1e-5)

3-element BitVector:
 1
 1
 1

#### Check: Eigenvectors
Let's do the same thing with the eigenvectors. How similar are our eigenvectors computed using [the `qriteration(...)` function](src/Compute.jl) to those calculated using [the `eigen(...)` method](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.eigen)?

In [19]:
i = 1; # which eigenvector do I want to check?
norm(V[:,i] - V̂[i])

9.497994605621654e-11

## OK, so we get the correct answer. But ... buy versus build?
__True backstory__: I was supposed to edit a paper of Aaron, a PhD student in the lab (an essential part of his thesis). I said I would do it over the weekend. However, instead I spent almost the entire weekend writing the `qriteration(...)` function in [src/Compute.jl](src/Compute.jl). Fast forward to Monday morning at the group meeting, Aaron excitedly asks me about the edits, and I have to explain that I got distracted by writing an implementation of the [QR iteration algorithm](https://en.wikipedia.org/wiki/QR_algorithm).
* __Was it worth it?__ Does our implementation of the [QR iteration algorithm](https://en.wikipedia.org/wiki/QR_algorithm) for computing eigenvalues and eigenvectors beat (in either time or memory) the [built-in eigen function](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.eigen)?

In [21]:
@benchmark eigen($A)

BenchmarkTools.Trial: 10000 samples with 9 evaluations.
 Range (min … max):  2.069 μs …  1.999 ms  ┊ GC (min … max): 0.00% … 99.65%
 Time  (median):     2.352 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.569 μs ± 19.962 μs  ┊ GC (mean ± σ):  7.75% ±  1.00%

                   ▃▇▂█▇▆▄                                    
  ▁▁▁▁▁▁▁▁▂▂▂▂▂▂▃▅█████████▆▄▄▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  2.07 μs        Histogram: frequency by time        2.86 μs <

 Memory estimate: 4.16 KiB, allocs estimate: 26.

In [22]:
@benchmark qriteration($A)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  16.250 μs …  1.515 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     17.417 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   19.221 μs ± 29.022 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

   ▃█▁▄                                                        
  ▄████▇▅▄▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂ ▃
  16.2 μs         Histogram: frequency by time        35.5 μs <

 Memory estimate: 16.16 KiB, allocs estimate: 269.